
<style>.text_cell .rendered_html * {direction: rtl; text-align: right;}</style>
                                    تمرین کامپیوتری ۱ 
                                      
                                     
                                       ایندیانا جونز
                                        
                                        تکتم سمیعی 
                                         
                                         810896054


<style>.text_cell .rendered_html * {direction: rtl; text-align: right;}</style>
 .هدف این پروژه حل یک معما به وسیله ی الگوریتم های سرچ در درس هوش مصنوعی است <br/>  
 در این مسئله باید از نقطه ی شروعی به انتهای انتهایی برویم و در این بین یک سری توپی که در جای نامناسبی قرار گرفته اند را ، سر جای خود قرار دهیم .<br/>
  تمام اطلاعات لازم به طور دقیق داده شده اند . 

<style>.text_cell .rendered_html * {direction: rtl; text-align: right;}</style>
در بخش زیر ، ابتدا نام فایل تکست مورد نظر را وارد کرده سپس اطلاعات لازم را از داخل فایل استخراج میکنیم .<br/>
در این قسمت از دستورات اسپلیت لاین و اسپلیت کمک میگیریم و اطلاعات را در تاپل های دوتای ذخیره میکنیم .<br/>
به همین صورت نقشه ی مسئله را استخرا میکنیم .

In [11]:
import math
import copy
import time
from IPython.display import HTML, display
# filename = input('Enter a file name: ')
f = open('3.txt', "r")
file = f.read()
line = file.splitlines()

map_size = line[0].split()
map_size = (int(map_size[0]),int(map_size[1]))

start_point = line[1].split()
start_point = (int(start_point[0]),int(start_point[1]))

end_point = line[2].split()
end_point = (int(end_point[0]),int(end_point[1]))

backpack_size = int(line[3])

number_of_balls = int(line[4])



mapp=[]
position_of_balls = {}
if number_of_balls != 0 :
    
    for i in range(number_of_balls):
        temp = line[i+5].split()
        position_of_balls[(int(temp[0]),int(temp[1]))]=(int(temp[2]),int(temp[3]))
    for i in range( len(line) - (4+number_of_balls+1) ):
        mapp.append(line[4+number_of_balls+1+i].split())
else :
    for i in range(5 , len(line)):
        mapp.append(line[i].split())

در این قسمت ، لیستی تحت عنوان استپ داریم که ۴ جهت بالا ، پایین ، چپ و راست را در خود دارد و برای اعمال ۴ جهت جغرافیایی به هر خانه است . در ادامه از این لیست برای پیمایش در نقشه استفاده میکنیم .

In [8]:
steps = [  (0,-1),(1,0),(0,1),(-1,0)]

برای شروع ، باید کلاسی تحت عنوان استیت داشته باشیم که در هر آبجکت آن سه دکشنری نگه داشته میشود . دیکشنری بالز نشان دهنده ی توپ های برداشته شده از زمین است . دیکشنری بک پک نشان میدهد چه توپ هایی در کوله پشتی ماست در حال حاضر و در دیکشنری اینسرتت بالز ، توپ های جابه جا شده را نگه میداریم . دو متغیر کاست و پوزیشن هم به ترتیب هزینه ی داده شده تا اینجای مسیر و موقعیت هر استیت در نقشه را نشان میدهد.<br/>
توابع ایک و هش را برای برای مشخص کردن پارامتر های مهم برای مساوی گرفتن دو آبجکت مجزا به صورت جدا تعریف کردیم . <br/> .
تابع گول چک میکند که اگر پوزیشن استیت با نقطه ی پایان یکی باشد و دیکشنری اینسرتت بالز شامل همه ی توپ های روی زمین باشد ، در این صورت ترو را برمیگرداند که نشان دهنده ی رسیدن به هدف است .<br/>
تابع اکسپند نقش اصلی را در این کلاس دارد . وظیفه ی آن چک کردن خانه های اطراف است و اگر میتواسنتیم به خانه ای اطراف برویم ، چک میکند در هرخانه امکان برداشتن توپ یا گذاشتن آن وجود دارد یا نه. سپس لیستی از خانه هایی مه میشد به آن ها رفت رابرمیگرداند . 

In [3]:
class State :

    capacity_of_backpack = backpack_size    

    def __init__(self,balls,inserted_balls, position,cost,backpack,parent):

        self.balls = balls
        self.inserted_balls = inserted_balls
        self.position = position
        self.cost = cost
        self.backpack = backpack
        self.parent = parent

    def __eq__(self,other):
        if (isinstance(other,State)):
            return self.position == other.position and self.balls == other.balls  and self.backpack == other.backpack

    def __hash__(self):
         return hash((str(self.position), str(self.balls)  , str(self.backpack)))


    def goal(self,end_point,number_of_balls):
        if (self.position == end_point):
            if number_of_balls ==0 :
                return True
            else :
                if (len(self.inserted_balls) == number_of_balls):
                    return True
        return False
 
    def print_parent(self):
        print("...............position :" , self.position , " cost :" , self.cost)
        if self.parent is not None : 
            return self.parent.print_parent()
        else:
            return
    def expand(self,position_of_balls):
        children = []
        for i in steps: 
            new_pos = tuple(map(sum,zip(self.position,i)))
            if (new_pos[0] >= 0) and (new_pos[0] <= map_size[0]-1) and (new_pos[1] >= 0) and (new_pos[1] <= map_size[1] -1) :
                if mapp[new_pos[0]][new_pos[1]] == "*":
                    
                    continue

                elif mapp[new_pos[0]][new_pos[1]] == "-":
                    BALLS = copy.deepcopy(self.balls)
                    INSERTED_BALLS = copy.deepcopy(self.inserted_balls)
                    COST = copy.deepcopy(self.cost)
                    BACKPACK = copy.deepcopy(self.backpack)
                    child = State(BALLS ,INSERTED_BALLS, new_pos ,COST + 1,BACKPACK ,self)
                    
                    if number_of_balls != 0 :

                        if  (len(self.backpack) < State.capacity_of_backpack) and (new_pos in position_of_balls) and (new_pos not in self.balls):
                            child.balls[new_pos] = position_of_balls[new_pos]
                            child.backpack[new_pos] = position_of_balls[new_pos]
                            child.cost += 1
                            children.append(child)
                            continue

                        key_list1 = list(self.backpack.keys())
                        val_list1 = list(self.backpack.values())
                        key_list2 = list(self.inserted_balls.keys())
                        val_list2 = list(self.inserted_balls.values())

                        if (new_pos in val_list1) and (new_pos not in val_list2):
                            positionn = val_list1.index(new_pos)
                            del child.backpack[key_list1[positionn]] 
                            child.inserted_balls[key_list1[positionn]] = self.balls[key_list1[positionn]]
                            children.append(child)
                            continue
                        else :
                            children.append(child)
                    else :
                        children.append(child)
                        continue
                            
            else :
                continue
        return children

در سرچ بی اف اس ، در ابتدا لیستی از فرانتیر نگه میداریم و و همچنین استیت های اکسپلور شده را در دیکشنری اکسپلورد نگه داری میکنیم . سپس یک استیت اولیه با داده های اولیه میسازیم. شرط هدف را برای استیت اولیه صدا میزنیم و در صورت هدف نبودن، استیت اولیه را به فرانتیر اضافه میکنیم و آن را در اکسپلورد ذخیره میکنیم.<br/>
در ادامه ، در داخل یک حلقه وایل ، از ابتدای لیست فرانتیر یک استیت پاپ میکنیم و فرزندان آن را با تابع اکسپند به دست میاوریم و شرط هدف را برای تک تک فرزندان تست میکنیم و یکی یکی فرزندان را به داخل فرانتیر اضافه میکنیم . در واقع استیتی که اول وارد میشود ، اول هم خارج میشود . در حقیقت فرانتیر در این الگوریتم مانند صف است یعنی در ابتدا همه ی بچه های یک نود چک میشوند سپس به بررسی لایه دوم فرزندان میرویم .

In [4]:
def BFS():
  
    frontier = []
    explored = {}
    
    sstate = State({},{},start_point, 0,{} ,None)
    frontCounter = 0
  
    if sstate.goal(end_point,number_of_balls):
        print('len explored : ', len(explored))
        print('num of pops from frontier: ', frontCounter)
        return sstate.cost
    frontier.append(sstate)
    explored[sstate.__hash__()] = True
    counter = 0
    while(True):

        if len(frontier) == 0:
            print("BBBB")
            print('len explored : ', len(explored))
            print('num of pops from frontier: ', frontCounter)
            return 'failier'
    
        currentState = frontier.pop(0)
        frontCounter += 1
    
        children = currentState.expand(position_of_balls)
    
        for i in range(len(children)):
            if children[i].__hash__() in explored:
                continue
        
            if children[i].goal(end_point,number_of_balls):
                print("the child is goal ")
                print('len explored : ', len(explored))
                print('num of pops from frontier: ', frontCounter)
                print("children.cost",children[i].cost)
                
                children[i].print_parent()
                return children[i].cost
        
            frontier.append(children[i])
            explored[children[i].__hash__()] = True

    

In [5]:

tic = time.time()
print(BFS())
tac = time.time()
print(tac - tic)

the child is goal 
len explored :  2303
num of pops from frontier:  2301
children.cost 48
...............position : (48, 48)  cost : 48
...............position : (47, 48)  cost : 47
...............position : (46, 48)  cost : 46
...............position : (45, 48)  cost : 45
...............position : (44, 48)  cost : 44
...............position : (43, 48)  cost : 43
...............position : (42, 48)  cost : 42
...............position : (41, 48)  cost : 41
...............position : (40, 48)  cost : 40
...............position : (39, 48)  cost : 39
...............position : (38, 48)  cost : 38
...............position : (37, 48)  cost : 37
...............position : (36, 48)  cost : 36
...............position : (35, 48)  cost : 35
...............position : (34, 48)  cost : 34
...............position : (33, 48)  cost : 33
...............position : (32, 48)  cost : 32
...............position : (31, 48)  cost : 31
...............position : (30, 48)  cost : 30
...............position : (29, 48)  

برای نمایش داده های مربوط به هر تست کیس تابع زیر را تعریف میکنیم :

In [61]:
def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))
data = [['  ','cost','len explored','num of pops from frontier','Average runtime'],['test 1',24,110,99,0.5],['test 2',48,2303,2301,0.18],['test 3',68,4508,4508,256],['test 4',92,456,456,0.1]]
display_table(data) 

IDS:

در الگوریتم آی دی اس یا آی دی دی اف اس ابتدا هر شاخه راتا عمق مشخص شده بررسی میکنیم سپس در صورت نیافتن هدف در عمق مشخص شده ، به عمق پایین تر میرویم ، یعنی فرزندان به صورت بی اف اس بررسی میکنیم . <br/>
در واقع فرانتیر در این الگوریتم یک استک است که هراستیتی که زودتر وارد شود ، دیرتر خارج میشود . البته این تابع به صورت بازگشتی طراحی شده است به گونه ای که تابع دی اف اس را برای عمق مشخصی صدا میزند و اگر به هدف نرسید ،عمق را افزایش میدهد و دوباره تابع بی اف اس را با عمق جدید و اکسپلورد خالی جدید صدا میزند . <br/>
در تابع بی اف اس در مرحله شرط هدف را چک میکند ، سپس اگز هدف نبود فرزندان را استخراج میکند و تابع بی اف اس را برای هر فرزند به صورت جدا با عمیق یکی کمتر صدا میزند . ماهیت بازگشتی این تابع به این گونه است که بعد از اتمام سرچ فرزندان یک شاخه ، به بررسی فرزندان قبلی ، به صورت گام به گام میپردازد . 


In [12]:
def IDDFS():
  
  d = 1
  
  sstate = State({},{},start_point, 0,{} ,None)
  
  while(True):
    explored = {}
    s = DFS(sstate, d, explored)
    
    if s != 'failure':
        print('len explored: ', len(explored))
        s.print_parent()
        return s.cost 
    
    d+=1

def DFS(state, depth, explored):
    if explored.get(state) != None and explored[state] >= depth :
        return 'failure'
  
    explored[state] = depth
  
    if state.goal(end_point,number_of_balls):
        return state
    
    if depth <= 0:
        return 'failure'

    children = state.expand(position_of_balls)
    for child in children:
        res = DFS(child, depth - 1, explored)
        if res is not 'failure':
            return res
    return 'failure'

In [ ]:
tic = time.time()
print(IDDFS())
tac = time.time()
print(tac - tic)

In [75]:
data = [['  ','cost','len explored','Average runtime'],['test 1',24,91,0.26],['test 2',48,624,25],['test 3',68,4508,256],['test 4',92,315,5]]
display_table(data) 

A* :

دو تابع هیوریستیک به صورت زیر تعریف میکنیم :<br/>
تابع اول :<br/>
این تابع به این صورت است که ابتدا شرط خالی بودن کوله پشتی را چک میکند ، در صورت خالی بودن ان فقط فاصله ی منهتن پوزیشن استیت با پوزیشن مقصد را محاسبه میکند . در صورت خالی نبودن کوله پشتی، فاصله ی پوزیشن استیت با مقصد هر توپی که داخل کوله پشتی است به علاوه ی فاصله ی استیت با هر توپی که روی زمین است و باید برداشته شود را محاسبه میکند . 

In [77]:
def heuristic1(state):
    if (len(state.inserted_balls) == number_of_balls):
        h =  (abs(state.position[0] - end_point[0]) + abs(state.position[1] - end_point[1]))
        return h
    else :
        h = 0 
        for ball in position_of_balls:
            if ball in state.backpack :
                 h += (abs(state.position[0]-state.backpack[ball][0])+abs(state.position[1]-state.backpack[ball][1]))
            else :
                h += (abs (ball[0]-position_of_balls[ball][0]) + abs (ball[1]-position_of_balls[ball][1]))
        return h          

تابع دوم : <br/>
در این تابع به صورت ساده فقط فاصله ی منهتن پوزیشن استیت با نقطه ی مقصد بررسی میشود .

In [89]:
def heuristic2(state):
#     print("&&&& state.position[0] ",state.position[0] , "^^^^^^^ state.position[1] ", state.position[1])
    h = (abs(state.position[0] - end_point[0]) + abs(state.position[1] - end_point[1]))
#     print("hhhh   ", h)
    return h

تابع ای استار همانند بی اف اس عمل میکند با این تفاوت که قبل از پاپ کردن استیت از فرانتیر، برای چک کردن فرزندانش ، استیتی پاپ میشود که مجموع تابع هیوریستیک و کاست کمتری داشته باشد . برای اینکار ، در هر دور از حلقه ی وایل ، بر روی تمام استیت های ذخیره شده در فرانیتر ، این مجموع را محاسبه میکنیم و در دیکشنری اف نگه میداریم ، سپس مینینم این مقدار را میابیم و اندیس آن را از لیست فرانتیر پاپ میکنیم .<br/>
ادامه ی مسیر مانند بی اف اس است .

In [97]:
def aStar():
    frontier = []
    explored = {}
  
    frontCounter = 0
    sstate = State({},{},start_point, 0,{} ,None)
  
    if sstate.goal(end_point,number_of_balls):
        print('len explored : ', len(explored))
        print('num of pops from frontier: ', frontCounter)
        return sstate.cost

    frontier.append(sstate)
    explored[sstate.__hash__()] = True
  
    while(True):
    
        if len(frontier) == 0:
            print('len explored : ', len(explored))
            print('num of pops from frontier: ', frontCounter)
            return 'failier'
    
        f = {}
        for i in range(len(frontier)):
            f[i] = (heuristic2(frontier[i]) + frontier[i].cost)
            
        currentState = frontier.pop(min(f, key=f.get))
        frontCounter += 1
        children = currentState.expand(position_of_balls)
    
        for i in range(len(children)):
            if children[i].__hash__() in explored:
                continue
                
            if children[i].goal(end_point,number_of_balls):
                print('len explored : ', len(explored))
                print('num of pops from frontier: ', frontCounter)
                return children[i].cost
        
            frontier.append(children[i])
            explored[children[i].__hash__()] = True

In [99]:
tic = time.time()
print(aStar())
tac = time.time()
print(tac - tic)

len explored :  110
num of pops from frontier:  98
24
0.02207207679748535


In [100]:
data = [['"heuristic1"','cost','len explored','num of pops from frontier','Average runtime'],['test 1',24,85, 74,0.02],['test 2',48,674,623,0.10],['test 3',68,4508,4508,256],['test 4',92,456,456,0.033]]
display_table(data) 
data2 = [['"heuristic2"','cost','len explored','num of pops from frontier','Average runtime'],['test 1',24,110, 98,0.025],['test 2',48,674,623,0.10],['test 3',68,4508,4508,256],['test 4',92,456,456,0.031]]
display_table(data2) 

همان طور که از نتایج مشاهده میکنید ، تعداد استیت های دیده شده به صورت چشمگیری نسبت به الگوریتم های قبلی کاهش مییابند ولی زمان ران برنامه ها تغییر محسوسی نمیابد .

Weighted A* :


در این الگوریتم ، دو تابه هیوریستیک رادر پارامتر الفا که بزرگ تر از یک است  ، ضرب مکنیم . اینکار باعث سرعت بخشیدن به پیدا کردن پاسخ میشود ولی امکان دارد دیگر تایع ما ادمیسبل نباشد نباشد ، یعنی مقدار هزینه ی تخمین زده شده ، از مقدار واقعی بیشتر باشد .<br/>
به ازای دو الفای مختلف این تابع را تست میکنیم :
در تابع اول ، مقدار اچ به دست آمده را در ۳ و در تابع دوم در ۲ ضرب میکنیم .

In [117]:
def heuristic1_(state):
    if (len(state.inserted_balls) == number_of_balls):
        h =  (abs(state.position[0] - end_point[0]) + abs(state.position[1] - end_point[1]))
        return h
    else :
        h = 0 
        for ball in position_of_balls:
            if ball in state.backpack :
                 h += (abs(state.position[0]-state.backpack[ball][0])+abs(state.position[1]-state.backpack[ball][1]))
            else :
                h += (abs (ball[0]-position_of_balls[ball][0]) + abs (ball[1]-position_of_balls[ball][1]))
        return 3*h 

In [123]:
def heuristic2_(state):
#     print("&&&& state.position[0] ",state.position[0] , "^^^^^^^ state.position[1] ", state.position[1])
    h = (abs(state.position[0] - end_point[0]) + abs(state.position[1] - end_point[1]))
#     print("hhhh   ", h)
    return 2*h

In [136]:
def weighted_aStar():
    frontier = []
    explored = {}
  
    frontCounter = 0
    sstate = State({},{},start_point, 0,{} ,None)
  
    if sstate.goal(end_point,number_of_balls):
        print('len explored : ', len(explored))
        print('num of pops from frontier: ', frontCounter)
        return sstate.cost

    frontier.append(sstate)
    explored[sstate.__hash__()] = True
  
    while(True):
    
        if len(frontier) == 0:
            print('len explored : ', len(explored))
            print('num of pops from frontier: ', frontCounter)
            return 'failier'
    
        f = {}
        for i in range(len(frontier)):
            f[i] = (heuristic1_(frontier[i]) + frontier[i].cost)
            
        currentState = frontier.pop(min(f, key=f.get))
        frontCounter += 1
        children = currentState.expand(position_of_balls)
    
        for i in range(len(children)):
            if children[i].__hash__() in explored:
                continue
                
            if children[i].goal(end_point,number_of_balls):
                print('len explored : ', len(explored))
                print('num of pops from frontier: ', frontCounter)
                return children[i].cost
        
            frontier.append(children[i])
            explored[children[i].__hash__()] = True

In [138]:
tic = time.time()
print(weighted_aStar())
tac = time.time()
print(tac - tic)

len explored :  456
num of pops from frontier:  456
92
0.033033132553100586


In [139]:
data = [['"heuristic1_"','cost','len explored','num of pops from frontier','Average runtime'],['test 1',24,79, 66,0.013],['test 2',48,674,623,0.13],['test 3',68,4508,4508,256],['test 4',92,456,456,0.033]]
display_table(data) 
data2 = [['"heuristic2_"','cost','len explored','num of pops from frontier','Average runtime'],['test 1',24,109, 102,0.023],['test 2',48,120,48,0.006],['test 3',68,4508,4508,256],['test 4',92,306,299,0.021]]
display_table(data2) 

همانطور که مشاهده تاثیر تایع هیوریستیک دوم در تست ۲ بسیار محسوس است چون ایت تست هیچ گویی ندارد .و همچنین به صورت کلی نسبت به الگوریتم ای استار تعداد استیت ها و زمان ران کاهش یافته است . 

نتیجه گیری :
در این تمرین الگوریتم های سرچ مختلف را بررسی کردیم . با توجه به نتایج ،الگوریتم های سرچ آگاهانه با هزینه کمتری و در مدت زمان کوتاه تری مارا به پاسخ هدایت میکنند ، به شرط تعریف درست و منطقی توابع هیوریستیک. در صورت پیچیدگی بالای توابع هیوریستیک ، زمان ران برنامه افزایش میابد . 